In [ ]:
import sys
sys.path.append("../")
from aoc_utils import *

<a id='home'></a>
Each day's work will consist of three tasks:
- **Input**: Parse the day's input file with the function `parse(day, parser, sep)`, which treats the input as a sequence of *entries*, separated by `sep` (default newline); applies `parser` to each entry; and returns the results as a tuple. (Note: `ints` and `atoms` are useful `parser` functions (as are `int` and `str`).)
- **Part 1**: Write code to compute the answer to Part 1, and submit the answer to the AoC site. Use the function `answer` to record the correct answer and serve as a regression test when I re-run the notebook.
- **Part 2**: Repeat coding and `answer` for Part 2.
<br>
1. [Day 1](#day_1)
2. [Day 2](#day_2)
3. [Day 3](#day_3)
4. [Day 4](#day_4)
5. [Day 5](#day_5)
6. [Day 6](#day_6)
7. [Day 7](#day_7)
8. [Day 8](#day_8)
9. [Day 9](#day_9)
10. [Day 10](#day_10)

<a id='day_1'></a>
[home](#home)
# [Day 1](https://adventofcode.com/2021/day/1): Sonar Sweep

count the number of times a depth measurement increases from the previous measurement


In [ ]:
get_in_file(1)
in1 = parse(1, int)
incr = Counter([1 if b > a else -1 for a, b in pairwise(in1)])[1] 
submit(incr, part="a", day=1, year=2021)

### Part Two

consider sums of a three-measurement sliding window

In [ ]:
sw = [c+b+a for a,b,c in sliding_window(in1, 3)] 
incr = Counter([1 if b > a else -1 for a,b in pairwise(sw)])[1]
submit(incr, part="b", day=1, year=2021)

<a id='day_2'></a>
[home](#home)
# [Day 2](https://adventofcode.com/2021/day/2): Dive!

- forward X increases the horizontal position by X units.
- down X increases the depth by X units.
- up X decreases the depth by X units.


In [ ]:
get_in_file(2)
parse_in = lambda x : { k:int(v) for (k,v) in [str.split(x)] }
in2 = Input(2, parse_in)

In [ ]:
c = Counter()
[c.update(d) for d in in2]; c

In [ ]:
course = c['forward']*(c['down']-c['up'])
submit(course, part="a", day=2, year=2021)

### Part Two

- down X increases your aim by X units.
- up X decreases your aim by X units.
- forward X does two things:
    - It increases your horizontal position by X units.
    - It increases your depth by your aim multiplied by X.

In [ ]:
aim = 0
horiz = 0
depth = 0
for d in in2 :
    for k,v in d.items() :
        if k == 'down':
            aim += v
        if k == 'up':
            aim -= v
        if k == 'forward':
            horiz += v
            depth += aim * v

aim, horiz, depth
submit(horiz*depth, part="b", day=2, year=2021)

<a id='day_3'></a>
[home](#home)
# [Day 3](https://adventofcode.com/2021/day/3): Binary Diagnostic

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used

The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

In [ ]:
get_in_file(3)
parse_in = lambda x : mapt(int, list(str.strip(x)))
in3 = Input(3, parse_in)

In [ ]:
inT = np.array(in3).T
print(f"input Transposed shape {inT.shape}")
bits_counter = mapt( lambda x : Counter(x.tolist()).most_common(), inT ); bits_counter

In [ ]:
gamma_rate_bits = mapt( lambda x : Counter(x.tolist()).most_common()[0][0], inT )
print (f"gamma_rate_bits {gamma_rate_bits}")
gamma_rate = int("".join(map(str,gamma_rate_bits)), 2); gamma_rate

In [ ]:
epsilon_rate_bits = mapt( lambda x : Counter(x.tolist()).most_common()[-1][0], inT )
print (f"epsilon_rate_bits {epsilon_rate_bits}")
epsilon_rate = int("".join(map(str,epsilon_rate_bits)), 2); epsilon_rate

In [ ]:
submit(gamma_rate*epsilon_rate, part="a", day=3, year=2021)

### Part Two

The bit criteria depends on which type of rating value you want to find:

- To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
- To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

In [ ]:

def Oxy_bit_criteria(x) :
    c = Counter(x.tolist())
    #print (c)
    if c[0] == c[1]:
        return 1
    return c.most_common()[0][0]

def CO2_bit_criteria(x) :
    c = Counter(x.tolist())
    #print (c)
    if c[0] == c[1]:
        return 0
    return c.most_common()[-1][0]

def calc_rate(input, bit_fun_criteria) :

    col = 0
    inM = np.array(input)

    while ( inM.shape[0] > 1 ) :
        # find criteria bit in column "col"
        criteria_bit = mapt( bit_fun_criteria, inM.T )[col]
        #print (f"col {col} criteria bit {criteria_bit}")
        # get a boolean vector which select rows that have "most_common_bit" at column "col" 
        row_selector = inM[:,col] == criteria_bit
        # filter all the rows using boolean vector "row_selector" 
        inM = inM[row_selector,:]
        col += 1

    return inM[0]

In [ ]:
oxygen_generator_rating = int("".join(map(str, calc_rate(in3, Oxy_bit_criteria))), 2); oxygen_generator_rating

In [ ]:
co2_scrubber_rating = int("".join(map(str, calc_rate(in3, CO2_bit_criteria))), 2); co2_scrubber_rating

In [ ]:
submit(oxygen_generator_rating*co2_scrubber_rating, part="b", day=3, year=2021)

<a id='day_4'></a>
[home](#home)
# [Day 4](https://adventofcode.com/2021/day/4): Giant Squid

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

In [ ]:
get_in_file(4)

in4 = parse(4, ints, sep="\n\n")

def bingo_score(input) :
    order, *boards = input
    boardsM = mapt(lambda b: np.array(b).reshape(5,5), boards)
    boardsK = mapt(lambda b: np.full((5,5), False), boards)
    it_order = iter(order)
    tmpK = list()
    drawn = list()
    bingo = False

    while not bingo :
        draw = next(it_order)
        for bM,bK in zip(boardsM, boardsK) :
            # mask elem is True where "draw" is in bM
            mask = np.isin(bM, draw)
            bK = bK | mask
            # If all numbers in any row or any column of a board are marked
            # that board wins
            if ( np.any(np.all(bK, axis=0)) or 
                 np.any(np.all(bK, axis=1)) ) :
                bingo = True
                break
            
            tmpK.append(bK)
        #
        drawn.append(draw)
        boardsK = tmpK
        tmpK = list()

    return (draw,drawn,bM,bK)

d,D,M,K = bingo_score(in4)
print (M)
#print (D)
#print (K)
#print (np.logical_not(K))
score = np.sum(M[np.logical_not(K)])*d


In [ ]:
submit(score, part="a", day=4, year=2021)

### Part Two

... figure out which board will win last

In [ ]:
def win_boards(input) :
    order, *boards = input
    boardsM = mapt(lambda b: np.array(b).reshape(5,5), boards)
    boardsK = mapt(lambda b: np.full((5,5), False), boards)
    it_order = iter(order)
    tmpK = list()
    tmpM = list()
    drawn = list()
    bingo = False
    bingo_boards = list()

    while True :
        try :
            draw = next(it_order)
            for bM,bK in zip(boardsM, boardsK) :
                # mask elem is True where "draw" is in bM
                mask = np.isin(bM, draw)
                bK = bK | mask
                # If all numbers in any row or any column of a board are marked
                # that board wins
                if ( np.any(np.all(bK, axis=0)) or 
                    np.any(np.all(bK, axis=1)) ) :
                    bingo_boards.append((draw,bM,bK))
                else:
                    tmpK.append(bK)
                    tmpM.append(bM)
            #
            drawn.append(draw)
            boardsK = tmpK
            boardsM = tmpM
            tmpK = list()
            tmpM = list()
        except (StopIteration) :
            break

    return bingo_boards

d,M,K = win_boards(in4)[-1]

score = np.sum(M[np.logical_not(K)])*d

In [ ]:
submit(score, part="b", day=4, year=2021)

<a id='day_5'></a>
[home](#home)
# [Day 5](https://adventofcode.com/2021/day/5): Hydrothermal Venture

In [ ]:
get_in_file(5)

in5 = parse(5, ints)

In [ ]:
class Point(namedtuple('Point', ['x', 'y'])):
    pass

class Segment(namedtuple('Segment', ['p1', 'p2'])):
    def is_V(self):
        return self.p1.x == self.p2.x
    def is_H(self):
        return self.p1.y == self.p2.y
    def is_D(self):
        return (abs(self.p1.x - self.p2.x) == abs(self.p1.y - self.p2.y))
    def min_x(self):
        return min(self.p1.x ,self.p2.x)
    def min_y(self):
        return min(self.p1.y ,self.p2.y)
    def max_x(self):
        return max(self.p1.x ,self.p2.x)
    def max_y(self):
        return max(self.p1.y ,self.p2.y)
    def slope(self):
        return (self.p2.y - self.p1.y)/(self.p2.x - self.p1.x)
    def get_points(self):
        if ( self.is_V() ) :
            return mapt(lambda y: Point(self.p1.x,y), range(self.min_y(),self.max_y()+1))
        if ( self.is_H() ) :
            return mapt(lambda x: Point(x,self.p1.y), range(self.min_x(),self.max_x()+1))
        if ( self.is_D() ) :
            if self.slope() > 0 :
                return mapt(lambda z: Point(z[0],z[1]),
                            zip(range(self.min_x(),self.max_x()+1),
                                range(self.min_y(),self.max_y()+1)))
            else :
                return mapt(lambda z: Point(z[0],z[1]),
                            zip(range(self.max_x(),self.min_x()-1, -1),
                                range(self.min_y(),self.max_y()+1)))


In [ ]:
lines = [Segment(Point(x1,x2),Point(y1,y2)) for (x1,x2,y1,y2) in in5]
HV_lines = [line for line in lines if line.is_H() or line.is_V()]

In [ ]:
maxx = max([seg.max_x() for seg in HV_lines])
maxy = max([seg.max_y() for seg in HV_lines])
maxx,maxy

In [ ]:
pts_cnt = Counter()
mapt(lambda x: pts_cnt.update(Counter(x.get_points())), HV_lines)
# ... number of points where at least two lines overlap ==> >= 2
pts_num = len([v for v in pts_cnt.values() if v > 1])
pts_num = len([v for k,v in pts_cnt.most_common() if v > 1])
pts_num

In [ ]:
submit(pts_num, part="a", day=5, year=2021)

### Part Two

In [ ]:
test1 = [Segment(Point(1,1),Point(3,3)),Segment(Point(4,4),Point(2,2)),
         Segment(Point(0,1),Point(3,4)),Segment(Point(2,3),Point(0,1)),
         Segment(Point(9,7),Point(7,9))]
for t in test1 :
    t.is_D()
    print (t.slope(), t.get_points())


In [ ]:
D_lines = [line for line in lines if line.is_D()]
pts_cnt = Counter()
mapt(lambda x: pts_cnt.update(Counter(x.get_points())), chain(HV_lines,D_lines))
pts_num = len([v for k,v in pts_cnt.most_common() if v > 1])
pts_num

In [ ]:
submit(pts_num, part="b", day=5, year=2021)

<a id='day_6'></a>
[home](#home)
# [Day 6](https://adventofcode.com/2021/day/6): Lanternfish


In [ ]:
get_in_file(6)
in6 = parse(6, ints)

In [ ]:
init_state, = in6
init_test = 3,4,3,1,2

In [ ]:
def population (in_s, days, verbose=False) :
    curr_state = in_s 
    while days > 0 :
        if verbose :
            print (f"...days {days} len(curr_state) = {len(curr_state)}")
        c = Counter(curr_state)
        # Each day, a 0 becomes a 6 and adds a new 8 to the end of the list
        next_state = list(map(lambda x: 6 if x == 0 else x-1 , curr_state))
        # count 0s and append 8s as list
        #next_state += c[0]*[8]
        #next_state.extend(c[0]*[8])
        #curr_state = next_state
        # ------
        curr_state = next_state + c[0]*[8]
        #curr_state = [*next_state, *(c[0]*[8])]
        #curr_state = list(chain(next_state, c[0]*[8]))
        days -= 1
        
    return len(curr_state)

In [ ]:
days = 80
population_80d = population(init_state, days)
print (f"after {days} days there would be {population_80d} lanternfish" )
#days = 256; print (f"after {days} days {population(init_test, days)}" )

In [ ]:
cnt = population(init_state, 80)
submit(cnt, part="a", day=6, year=2021)
len(cnt)

### Part Two

How many lanternfish would there be after 256 days?

In [ ]:
%load_ext line_profiler

In [ ]:
def s1(S: list) -> list :
    '''using list comprehention '''
    return [ 6 if x == 0 else x-1 for x in S ]

def s2(S: list) -> list :
    ''' using list generator '''
    return list(( 6 if x == 0 else x-1 for x in S ))

def s3(S: list) -> list :
    ''' using trivial for '''
    res = list()
    for s in S :
        if s == 0 :
            res.append(6)
        else :
            res.append(s-1)
    return res 

def s4(S: list) -> Generator :
    ''' using generator '''
    return ( 6 if x == 0 else x-1 for x in S )

def sc(S: list) -> Iterator :
    ''' '''
    dq = deque()
    #print(f"split {len(S)}")
    for i,c in zip(count_from(),chunks(S,64)):
        #chunk = list(c)
        #print(f"{i}) {chunk} -> {s1(chunk)}")
        dq.append(s4(c))
    return chain.from_iterable(dq)

def population (in_s: list, days: int, S_func: callable =s1, verbose: bool =False) -> int :
    verbose = False
    curr_state = in_s
    print (f"S_func {S_func.__name__} {S_func.__doc__}" )
    while days > 0 :
        if verbose :
            print (f"...days {days} len(curr_state) = {len(curr_state)}")
        c = Counter(curr_state)
        # Each day, a 0 becomes a 6 and adds a new 8 to the end of the list
        #--- OLD ==> too much time for mem allocation/deallocation of huge lists ---#
        next_state = list(S_func(curr_state))
        # count 0s and append 8s list
        curr_state = next_state + c[0]*[8]
        days -= 1
        
    return len(curr_state)

In [ ]:
days = 80
#population2_80d = population(init_test, days, sc)
population2_80d = population(init_state, days, sc)
#assert(population2_80d==population_80d)
print (f"after {days} days there would be {population2_80d} lanternfishes" )

In [ ]:
def better_population (in_s: list, days: int, verbose: bool =False) -> int :

    cS = Counter(in_s)
    days = days
    cnt = 0
    while cnt < days :
        if verbose :
            print (f"day {cnt} {cS.most_common()}")
        # subtract 1 to all Counter keys 
        tmp = Counter()
        for k in cS.keys() :
            if k == 0 :
                tmp[6] += cS[k]
            else :
                tmp[k-1] += cS[k]
        if cS[0] :
            tmp[8] += cS[0]
        cS = tmp        
        cnt += 1

    return cS.total()


In [ ]:
days = 256
better_population_256d = better_population(init_state, days)
print (f"after {days} days there would be {better_population_256d} lanternfishes" )
submit(better_population_256d, part="b", day=6, year=2021)

In [ ]:
import os
pname = "lp_population_"
for d in range(140,145,5) :
    %lprun -T lp_file -f s3 -f population population(init_test,d,s4)
    os.rename('lp_file',pname+str(d))
    
%lprun -T lp_file -f better_population better_population(init_state,256)

<a id='day_7'></a>
[home](#home)
# [Day 7](https://adventofcode.com/2021/day/7): The Treachery of Whales

In [ ]:
get_in_file(7)
in7, = parse(7, ints)

In [ ]:
cntr = Counter(in7)
print (f"input size {len(in7)} \
  min,max {min(in7),max(in7)} \
  mean {mean(in7)} \
  median {median(in7)} \
  mode {mode(in7)} \
\n{len(cntr)} {cntr.most_common(10)}")

In [ ]:
for t in (mean(in7),median(in7),mode(in7)):
    print (sum([abs(round(x-t)) for x in in7]))

In [ ]:
fuel4median = sum([abs(round(x-median(in7))) for x in in7])
submit(fuel4median, part="a", day=7, year=2021)

### Part Two

In [ ]:
def best_horiz_pos(pos_list) :
    crab_cntr = Counter(pos_list)
    m,M = min(pos_list),max(pos_list)
    bhp = Counter()
    for target in range(m,M+1) : 
        for crab_pos, crabs in crab_cntr.most_common() :
            bhp[target] += sum(range(abs(target-crab_pos)+1))*crabs

    return bhp


In [ ]:
bhp = best_horiz_pos(in7)
best_crabs_horiz_pos,fuel = bhp.most_common()[-1]
submit(fuel, part="B", day=7, year=2021)

<a id='day_8'></a>
[home](#home)
# [Day 8](https://adventofcode.com/2021/day/8): Seven Segment Search

Each entry consists of ten unique signal patterns, a | delimiter, and finally the four digit output value

In [ ]:
get_in_file(8)
in8 = parse(8,lambda x: x.split("|"))
seg2num = dict(zip([6,2,5,5,4,5,6,3,7,6],list(range(10))))

In [ ]:
out_values = [str.split(y) for x,y in in8 ]

In [ ]:
out_values_2347 = [digit for display_values in out_values for digit in display_values if len(digit) in set({2,4,3,7})]

In [ ]:
submit(len(out_values_2347), part="a", day=8, year=2021)

### Part Two

In [ ]:
def sort_seg(seg :set):
    return  "".join(sorted(seg))

def rule_L6 (L3: set, L4: set, L6: list[set] ) :

    l6 = dict()
    # find 6 in L6 : for each seg_set in L6 if L3 not in seg_Set ==> 6
    for x in L6 :
        if not L3.issubset(x) :
            l6[6] = sort_seg(x)
            #print(x)
            L6.remove(x)
    #print (L6) 
    a,b = L6
    amb = a-b
    if ( amb.issubset(L4) ) :
        l6[9] = sort_seg(a)
        l6[0] = sort_seg(b)
    else :
        l6[0] = sort_seg(a)
        l6[9] = sort_seg(b)

    return l6


def rule_L5 (L2: set, L4: set, L5: list[set] ) :

    l5 = dict()
    # find 3 in L5 : for each seg_set in L5 if L2 in seg_Set ==> 3
    for x in L5 :
        if L2.issubset(x) :
            l5[3] = sort_seg(x)
            #print(x)
            L5.remove(x)
    #print (L5) 
    a,b = L5
    amb = a-b
    if ( amb.issubset(L4) ) :
        l5[5] = sort_seg(a)
        l5[2] = sort_seg(b)
    else :
        l5[2] = sort_seg(a)
        l5[5] = sort_seg(b)
        
    return l5


def display_inference(in_lines):

    pattern_output_lines = [(str.split(x),str.split(y)) for x,y in in_lines ]  
    all_data = list()
    sum_outputs = 0

    for patterns,outputs in pattern_output_lines :
        base = dict()
        other = dict()
        display = dict()
        for pattern in patterns :
            s_pattern = sort_seg(pattern)
            if ( len(s_pattern) in set({2,4,3,7}) ) :
                base[len(s_pattern)] = s_pattern
                display[seg2num[len(s_pattern)]] = s_pattern
            else :
                try : other[len(s_pattern)].append(s_pattern)
                except KeyError : other[len(s_pattern)] = [s_pattern]
        #
        L2 = set(base[2])
        L3 = set(base[3])
        L4 = set(base[4])
        L5 = list(map(set,other[5]))
        L6 = list(map(set,other[6]))
        display.update(rule_L6(L3,L4,L6))
        display.update(rule_L5(L2,L4,L5))
        
        dswap = dict((v,k) for k,v in display.items())
        out_sorted = list(map(sort_seg,outputs))
        sum_outputs = int("".join([str(dswap[osor]) for osor in out_sorted ]))
        
        #all_data.append((base,other,display,dswap,out_sorted))
        all_data.append((display,out_sorted,sum_outputs))
    
    return all_data


In [ ]:

res = 0
for d,ov,s in display_inference(in8) :
    res += s
res    

In [ ]:
submit(res, part="b", day=8, year=2021)

<a id='day_9'></a>
[home](#home)
# [Day 9](https://adventofcode.com/2021/day/9): Smoke Basin
Find the low points - the locations that are lower than any of its adjacent locations.<br>
Most locations have four adjacent locations (up, down, left, and right);<br>
locations on the edge or corner of the map have three or two adjacent locations, respectively.

In [ ]:
get_in_file(9)
p = lambda x: mapt(int,x)
in9 = parse(9,p)

test_in = """\
2199943210
3987894921
9856789892
8767896789
9899965678
"""

in_test = mapt(p,test_in.rstrip().split())


In [ ]:
def get_adj(i,j,M):
    """
          | i-1,j |
    ------------------------
    i,j-1 | i  ,j | i,j+1
    ------------------------
          | i+1,j |  
    """
    I,J = M.shape
    I -= 1
    J -= 1
    adj = {}
    if ( i == 0 ) :
        if ( j == 0) :
            adj[i,j+1] = M[i][j+1]
            adj[i+1,j] = M[i+1][j]
        elif ( j == J ) :
            adj[i+1,j] = M[i+1][j]
            adj[i,j-1] = M[i][j-1]
        else :
            adj[i,j+1] = M[i][j+1]
            adj[i+1,j] = M[i+1][j]
            adj[i,j-1] = M[i][j-1]
    elif ( i == I ) :
        if ( j == 0) :
            adj[i-1,j] = M[i-1][j]
            adj[i,j+1] = M[i][j+1]
        elif ( j == J ) :
            adj[i-1,j] = M[i-1][j]
            adj[i,j-1] = M[i][j-1]
        else :
            adj[i-1,j] = M[i-1][j]
            adj[i,j+1] = M[i][j+1]
            adj[i,j-1] = M[i][j-1]
    elif ( j == 0 ) :
            adj[i-1,j] = M[i-1][j]
            adj[i,j+1] = M[i][j+1]
            adj[i+1,j] = M[i+1][j]
    elif ( j == J ) :
            adj[i-1,j] = M[i-1][j]
            adj[i+1,j] = M[i+1][j]
            adj[i,j-1] = M[i][j-1]
    else :
            adj[i-1,j] = M[i-1][j]
            adj[i,j+1] = M[i][j+1]
            adj[i+1,j] = M[i+1][j]
            adj[i,j-1] = M[i][j-1]

    return adj

In [ ]:
def get_low_points(inP):
    M = np.array(inP)
    print(M.shape)
    minSet = set()
    for ij in np.ndindex(M.shape):
        adj = get_adj(*ij,M)
        if min(adj.values()) > M[ij] :
            # remove any adj(i,j) points from minSet
            minSet.difference_update(adj.keys())
            # add (i,j) point to minSet
            minSet.add(ij)
    low_points = dict()
    for pt in minSet :
        low_points[pt]  = M[pt]
    return low_points



In [ ]:
def risk_level(lps: dict) -> int :
    return sum(mapt(lambda x: x+1, lps.values()))

In [ ]:
lp = get_low_points(in9)
print(lp)
risk = risk_level(lp)
risk

In [ ]:
submit(risk, part="a", day=9, year=2021)

### Part Two

A basin is all locations that eventually flow downward to a single low point.<br>
Therefore, every low point has a basin

In [ ]:
def get_basin_points(inP, pt: tuple) -> set:
    M = np.array(inP)
    tmpSet = set()
    tmpSet.add(pt)
    basinSet = set()
    basinSet.add(pt)
    while len(tmpSet) :
        pt = tmpSet.pop()
        adj = get_adj(*pt,M)
        for pt in adj :
            if M[pt] < 9 and pt not in basinSet :
                #print(f"add {pt}")
                basinSet.add(pt)
                tmpSet.add(pt)
    return basinSet
        

In [ ]:
get_basin_points(in_test, (0,1))

In [ ]:
for lp in get_low_points(in_test).keys() :
    bp = get_basin_points(in_test, lp)
    print(len(bp),bp)

In [ ]:
bp_len = list()
for lp in get_low_points(in9).keys() :
    bp_len.append(len(get_basin_points(in9, lp)))
bp_len.sort()
bp_len[-3:]


In [ ]:
submit(np.prod(bp_len[-3:]), part="b", day=9, year=2021)

<a id='day_10'></a>
[home](#home)
# [Day 10](https://adventofcode.com/2021/day/10): Syntax Scoring

In [109]:
get_in_file(10)
#p = lambda x: mapt(int,x)
in10 = parse(10)